In [271]:
imagename = '4YIO'
#imagename = 'FBAP'
imagename = 'PI17'

# https://www.interasia.cc/content/c_service/cargo_tracking.aspx?SiteID=1

# Load Image (Palette)

In [272]:
from PIL import Image, ImageFilter, ImageEnhance
from IPython.display import display
import pytesseract as tess
import os
from collections import Counter

ims = []
dirname = os.path.join(os.getcwd(), imagename)
for filename in os.listdir(dirname) :
    fullname = os.path.join(dirname, filename)
    #print(fullname)
    im = Image.open(fullname)
    display(im)
    ims.append(im)

# Convert to RGB

In [273]:
ims_rgb = []
for im in ims :
    im_rgb = im.convert("RGB")
    #display(im_rgb)
    ims_rgb.append(im_rgb)

# pixelwise median image

In [274]:
def median_image ( ims_rgb ) :
    from statistics import median
    
    im_processed = Image.new("RGB", ims[0].size, (255, 255, 255))
    pix_processed = im_processed.load()
    
    for x in range(1,im_processed.size[0]-1) :
        for y in range(1,im_processed.size[1]-1) :
            
            rs = []
            gs = []
            bs = []
            for im in ims_rgb :
                rgb = im.load()[x,y]
                #print(rgb)
                rs.append(rgb[0])
                gs.append(rgb[1])
                bs.append(rgb[2])
            
            r = int(median(rs))
            g = int(median(gs))
            b = int(median(bs))
        
            pix_processed[x,y] = ( r, g, b )
    
    return im_processed

# image-wise 9-median, pixelwise median

In [275]:
def median_image_single ( ims_rgb ) :
    from statistics import median
    
    def pattern ( pix, x, y ) :
        def indexed ( pix, x, y, idx ) :
            return pix[x,y][idx]
        r = []
        g = []
        b = []
        for dx in [-1,0,1] :
            for dy in [-1,0,1] :
                r.append( pix[x+dx,y+dy][0] )
                g.append( pix[x+dx,y+dy][1] )
                b.append( pix[x+dx,y+dy][2] )
        return (median( r ), median(g), median(b) )
    
    im_processed = Image.new("RGB", ims[0].size, (255, 255, 255))
    pix_processed = im_processed.load()
    
    for x in range(1,im_processed.size[0]-1) :
        for y in range(1,im_processed.size[1]-1) :
            
            rs = []
            gs = []
            bs = []
            for im in ims_rgb :
                rgb = pattern(im.load(),x,y) # im.load()[x,y]
                #print(rgb)
                rs.append(rgb[0])
                gs.append(rgb[1])
                bs.append(rgb[2])
            
            r = int(median(rs))
            g = int(median(gs))
            b = int(median(bs))
            grey = (r+g+b)/3
            mi = min(r,g,b)
            ma = max(r,g,b)
      
            pix_processed[x,y] = ( r, g, b )
    
    return im_processed

# stack-wise 9xN-median

In [276]:
def median_image_stack ( ims_rgb, stencil ) :
    from statistics import median
    
    def pattern ( pix, x, y ) :
        def indexed ( pix, x, y, idx ) :
            return pix[x,y][idx]
        r = []
        g = []
        b = []
        for dx in range(-stencil,stencil) :
            for dy in range(-stencil,stencil) :
                r.append( pix[x+dx,y+dy][0] )
                g.append( pix[x+dx,y+dy][1] )
                b.append( pix[x+dx,y+dy][2] )
        return [r, g, b]
    
    im_processed = Image.new("RGB", ims[0].size, (255, 255, 255))
    pix_processed = im_processed.load()
    
    for x in range(1+stencil,im_processed.size[0]-stencil+1) :
        for y in range(1+stencil,im_processed.size[1]-stencil+1) :
            
            rs = []
            gs = []
            bs = []
            for im in ims_rgb :
                rgb = pattern(im.load(),x,y) # im.load()[x,y]
                #print(rgb)
                rs.extend(rgb[0])
                gs.extend(rgb[1])
                bs.extend(rgb[2])
            
            r = int(median(rs))
            g = int(median(gs))
            b = int(median(bs))
      
            pix_processed[x,y] = ( r, g, b )
    
    return im_processed

# grey filter

In [277]:
def grey_filter( im, lightness, colorness ) :

    im_processed = Image.new("RGB", ims[0].size, (255, 255, 255))
    pix_processed = im_processed.load()
    
    for x in range(1,im_processed.size[0]) :
        for y in range(1,im_processed.size[1]) :
         
            rgb = im.load()[x,y]
            r = rgb[0]
            g = rgb[1]
            b = rgb[2]
            
            # light color
            if (r+g+b)/3 > lightness :
                r = g = b = 255
            # shade of grey
            if max(r,g,b)-min(r,g,b) < colorness :
                r = g = b = 255             
                
            pix_processed[x,y] = ( r, g, b )
            
    return im_processed

In [278]:
def grey_filter_hsv( im, minsaturation, maxvolumen ) :

    im_processed = Image.new("RGB", ims[0].size, (255, 255, 255))
    pix_processed = im_processed.load()
    
    im_hsv = im.convert('HSV')
    pix_hsv = im_hsv.load()
    
    for x in range(1,im_processed.size[0]) :
        for y in range(1,im_processed.size[1]) :
         
            rgb = im.load()[x,y]
            r = rgb[0]
            g = rgb[1]
            b = rgb[2]
            
            # not enough saturation
            if pix_hsv[x,y][1] < minsaturation :
                r = g = b = 255
                
            # too much volumen
            if pix_hsv[x,y][2] > maxvolumen :
                r = g = b = 255             
                
            pix_processed[x,y] = ( r, g, b )
            
    return im_processed

# tesseract call

In [279]:
def image_2_text( im ):
    
    def call_tesseract( im, psm ) :
        return tess.image_to_string(im, config='--psm ' + str(psm)).strip() # , lang='eng'
    
    recognized = []
    recognized.append(call_tesseract(im,6))
    recognized.append(call_tesseract(im,7))
    recognized.append(call_tesseract(im,8))
    recognized.append(call_tesseract(im,13))
    return Counter(recognized)

In [281]:
recognized = Counter()

for minsaturation in (30,50,70,90) :
    im_processed = median_image_stack( ims_rgb , 1 )
    im_processed = grey_filter_hsv ( im_processed, minsaturation, 255 )
    
    display(im_processed)
    text = image_2_text( im_processed )
    recognized += text
    print(text)
    
print("\nTotals: " + str(recognized))

Counter({'PIT7': 4})


Counter({'PI17': 2, 'PIT': 2})


Counter({'PI17': 2, 'PII7': 2})


Counter({'PI1T7': 2, 'PITT': 2})

Totals: Counter({'PIT7': 4, 'PI17': 4, 'PIT': 2, 'PII7': 2, 'PI1T7': 2, 'PITT': 2})


In [ ]:
#im_processed = remove_dots(im_processed, 150)
#display(im_processed)

im_processed = im_processed.filter(ImageFilter.MedianFilter(3))
display(im_processed)

print(image_2_text( im_processed ))

im_tess = Image.new("RGB", im_processed.size, (255, 255, 255))
im_tess.paste(im_processed)
#im_tess = im_tess.filter(ImageFilter.MedianFilter(1))
im_tess = im_tess.convert('1')
display(im_tess)

print(image_2_text( im_tess ))

# Process 5-Pixel-Average 
(should be applied 2 times)

In [175]:
def remove_dots ( image, threshold ) :
    from collections import Counter
    
    im_grey = im_rgb.convert("L")
    pix_grey = im_grey.load()
    
    im_processed = Image.new("RGB", im_grey.size, (255, 255, 255))
    pix_processed = im_processed.load()
    #threshold = 150

    for x in range(0,im_grey.size[0]) :
        pix_processed[x,0] = (255,255,255)
        pix_processed[x,im_grey.size[1]-1] = (255,255,255)
    for y in range(0,im_grey.size[1]) :
        pix_processed[0,y] = (255,255,255)
        pix_processed[im_grey.size[0]-1,y] = (255,255,255)

    for x in range(1,im_grey.size[0]-1) :
        for y in range(1,im_grey.size[1]-1) :
            
            checkval5 = (pix_grey[x-1,y] + pix_grey[x,y-1] + pix_grey[x,y] + pix_grey[x+1,y] + pix_grey[x,y+1]) / 5
            checkval9 = (pix_grey[x-1,y] + pix_grey[x,y-1] + pix_grey[x,y] + pix_grey[x+1,y] + pix_grey[x,y+1]
                        +pix_grey[x-1,y-1] + pix_grey[x+1,y-1] + pix_grey[x-1,y+1] + pix_grey[x+1,y+1] ) / 9
            
            checkval = checkval9
            if checkval > threshold :
                pix_processed[x,y] = (255,255,255)
            else :
                pix_processed[x,y] = (1,1,1) #(int(checkval),int(checkval),int(checkval))

    display(im_processed)
    #print("recognized  6 : '" + tess.image_to_string(im_processed, lang='eng', config='--psm 6') + "'")
    #print("recognized  7 : '" + tess.image_to_string(im_processed, lang='eng', config='--psm 7') + "'")
    #print("recognized  8 : '" + tess.image_to_string(im_processed, lang='eng', config='--psm 8') + "'")
    #print("recognized 13 : '" + tess.image_to_string(im_processed, lang='eng', config='--psm 13') + "'")
    
    recognized = []
    recognized.append(tess.image_to_string(im_processed, lang='eng', config='--psm 6').strip())
    recognized.append(tess.image_to_string(im_processed, lang='eng', config='--psm 7').strip())
    recognized.append(tess.image_to_string(im_processed, lang='eng', config='--psm 8').strip())
    recognized.append(tess.image_to_string(im_processed, lang='eng', config='--psm 13').strip())
    
    im_tess = Image.new("RGB", im_processed.size, (255, 255, 255))
    im_tess.paste(im_processed)
    im_tess = im_tess.filter(ImageFilter.MedianFilter(3))
    im_tess = im_tess.convert('1')
    display(im_tess)
    #print("recognized  6 : '" + tess.image_to_string(im_tess, lang='eng', config='--psm 6') + "'")
    #print("recognized  7 : '" + tess.image_to_string(im_tess, lang='eng', config='--psm 7') + "'")
    #print("recognized  8 : '" + tess.image_to_string(im_tess, lang='eng', config='--psm 8') + "'")
    #print("recognized 13 : '" + tess.image_to_string(im_tess, lang='eng', config='--psm 13') + "'")

    recognized.append(tess.image_to_string(im_tess, lang='eng', config='--psm 6').strip())
    recognized.append(tess.image_to_string(im_tess, lang='eng', config='--psm 7').strip())
    recognized.append(tess.image_to_string(im_tess, lang='eng', config='--psm 8').strip())
    recognized.append(tess.image_to_string(im_tess, lang='eng', config='--psm 13').strip())
    
    print( Counter(recognized) )
    return im_processed 

In [79]:
threshold = 130
im_processed = process(im_grey, threshold)
im_processed = process(im_processed.convert("L"), threshold)
im_processed = process(im_processed.convert("L"), threshold)
im_processed = process(im_processed.convert("L"), threshold)

Counter({'YDLW': 4, 'YD WwW': 2, 'VYDLW': 2})


Counter({'YDLW': 6, 'VYDLW': 2})


Counter({'YDLW': 4, 'VYDLW': 2, 'VYDiWw': 2})


Counter({'YDLW': 4, 'VYDiWw': 2, 'VYDiW': 2})
